In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup
import urllib.request, urllib.parse, urllib.error
import matplotlib.pyplot as plt
import warnings
from fredapi import Fred
from ta import add_all_ta_features
warnings.filterwarnings('ignore')

from warnings import simplefilter

warnings.filterwarnings('ignore')

def tech_analysis_spx():
    data = yf.download(
                # tickers list or string as well
                tickers = '^GSPC',

                # use "period" instead of start/end
                # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                # (optional, default is '1mo')
                period = "max",

                # fetch data by interval (including intraday if period < 60 days)
                # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                # (optional, default is '1d')
                interval = "1d",

                # group by ticker (to access via data['SPY'])
                # (optional, default is 'column')
                group_by = 'ticker',

                # adjust all OHLC automatically
                # (optional, default is False)
                auto_adjust = True,

                # download pre/post regular market hours data
                # (optional, default is False)
                prepost = True,

                # use threads for mass downloading? (True/False/Integer)
                # (optional, default is True)
                threads = True,

                # proxy URL scheme use use when downloading?
                # (optional, default is None)
                proxy = None
            )

    df = data.reindex(index=data.index[::-1])

    df = add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)

    df = df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)
    
    return df

def tech_analysis_10yr():
    data = yf.download(
                # tickers list or string as well
                tickers = '^TNX',

                # use "period" instead of start/end
                # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                # (optional, default is '1mo')
                period = "max",

                # fetch data by interval (including intraday if period < 60 days)
                # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                # (optional, default is '1d')
                interval = "1d",

                # group by ticker (to access via data['SPY'])
                # (optional, default is 'column')
                group_by = 'ticker',

                # adjust all OHLC automatically
                # (optional, default is False)
                auto_adjust = True,

                # download pre/post regular market hours data
                # (optional, default is False)
                prepost = True,

                # use threads for mass downloading? (True/False/Integer)
                # (optional, default is True)
                threads = True,

                # proxy URL scheme use use when downloading?
                # (optional, default is None)
                proxy = None
            )

    df = data.reindex(index=data.index[::-1])

    df = add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)

    df = df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)
    
    return df

def pull_tickers():
    tickers = ['^VIX', '^VVIX', '^GSPC', '^SKEW', 'CL=F', 'HG=F', 'GC=F']
    
    df = yf.download(tickers=tickers,
                # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                # (optional, default is '1mo')
                period = "max",

                # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                # (optional, default is '1d')
                interval = "1d",

                # group by ticker (to access via data['SPY'])
                # (optional, default is 'column')
                group_by = 'ticker',

                # adjust all OHLC automatically
                # (optional, default is False)
                auto_adjust = True,

                # download pre/post regular market hours data
                # (optional, default is False)
                prepost = True,

                # use threads for mass downloading? (True/False/Integer)
                # (optional, default is True)
                threads = True,

                # proxy URL scheme use use when downloading?
                # (optional, default is None)
                proxy = None
            )

    #df = data.reindex(index=data.index[::-1])
    df = df.drop(['Open', 'High', 'Low', 'Volume'], axis=1, level=1)
    df.columns = df.columns.map(' '.join).str.strip()
    df.columns = [x.split()[0] for x in df.columns]
    df = df.sort_index(axis=0, ascending=True)
    df = df.fillna(method='ffill')
    df = df.rename(columns={'^GSPC': 'SPX'})
    
    return df

def RSI(ticker):
    df = yf.download(tickers=ticker,
                # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                # (optional, default is '1mo')
                period = "max",

                # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                # (optional, default is '1d')
                interval = "1d",

                # group by ticker (to access via data['SPY'])
                # (optional, default is 'column')
                group_by = 'ticker',

                # adjust all OHLC automatically
                # (optional, default is False)
                auto_adjust = True,

                # download pre/post regular market hours data
                # (optional, default is False)
                prepost = True,

                # use threads for mass downloading? (True/False/Integer)
                # (optional, default is True)
                threads = True,

                # proxy URL scheme use use when downloading?
                # (optional, default is None)
                proxy = None
            )


    df.columns = ['Open', 'High', 'Low', 'Adj Close', 'Volume']

    ## 14_Day RSI
    df['Up Move'] = np.nan
    df['Down Move'] = np.nan
    df['Average Up'] = np.nan
    df['Average Down'] = np.nan
    # Relative Strength
    df['RS'] = np.nan
    # Relative Strength Index
    df['RSI'] = np.nan
    ## Calculate Up Move & Down Move
    for x in range(1, len(df)):
        df['Up Move'][x] = 0
        df['Down Move'][x] = 0

        if df['Adj Close'][x] > df['Adj Close'][x-1]:
            df['Up Move'][x] = df['Adj Close'][x] - df['Adj Close'][x-1]

        if df['Adj Close'][x] < df['Adj Close'][x-1]:
            df['Down Move'][x] = abs(df['Adj Close'][x] - df['Adj Close'][x-1])  

    ## Calculate initial Average Up & Down, RS and RSI
    df['Average Up'][14] = df['Up Move'][1:15].mean()
    df['Average Down'][14] = df['Down Move'][1:15].mean()
    df['RS'][14] = df['Average Up'][14] / df['Average Down'][14]
    df['RSI'][14] = 100 - (100/(1+df['RS'][14]))
    ## Calculate rest of Average Up, Average Down, RS, RSI
    for x in range(15, len(df)):
        df['Average Up'][x] = (df['Average Up'][x-1]*13+df['Up Move'][x])/14
        df['Average Down'][x] = (df['Average Down'][x-1]*13+df['Down Move'][x])/14
        df['RS'][x] = df['Average Up'][x] / df['Average Down'][x]
        df['RSI'][x] = 100 - (100/(1+df['RS'][x]))

    df['RSI SMA'] = df['RSI'].rolling(window=7).mean()
    
    return df['RSI SMA']

def VVIX():
    ticker_list = ['^VIX', '^VVIX']

    data = yf.download(
        # tickers list or string as well
        tickers = ticker_list[0:],

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "max",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

    df = data.reindex(index=data.index[::-1])
    df = df.drop(['Open', 'High', 'Low', 'Volume'], axis=1, level=1)
    df.columns = df.columns.map(' '.join).str.strip()
    df.columns = [x.split()[0] for x in df.columns]
    df = df.sort_index(axis=0, ascending=True)
    df = df.fillna(method='ffill')
    df['vvix_norm'] = (df['^VVIX'] - df['^VVIX'].mean())/df['^VVIX'].std()
    df['vix_norm'] = (df['^VIX'] - df['^VIX'].mean())/df['^VIX'].std()
    df['vvix_vix_minus'] = df['vvix_norm'] - df['vix_norm']
    df['vvix_vix_minus_norm'] = (df['vvix_vix_minus'] - df['vvix_vix_minus'].mean())/df['vvix_vix_minus'].std()
    df['vvix_vix_minus_abs'] = abs(df['vvix_norm'] - df['vix_norm'])
    df['vvix_vix_minus_abs_norm'] = (df['vvix_vix_minus_abs'] - df['vvix_vix_minus_abs'].mean())/df['vvix_vix_minus_abs'].std()
    df['vvix_vix_div'] = df['vvix_norm'] / df['vix_norm']
    df['vvix_vix_div_norm'] = (df['vvix_vix_div'] - df['vvix_vix_div'].mean())/df['vvix_vix_div'].std()
    df['vix_diff_sma'] = df['vvix_vix_minus_abs_norm'].rolling(window=10).mean()
    
    return df['vix_diff_sma']

def spx_corr():
    url = r'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    page = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(page, "html.parser")
    table = soup.find_all("table")
    ticker_df = pd.read_html(str(table[0]))
    ticker_df = pd.DataFrame(ticker_df[0])
    ticker_list = [x for x in ticker_df['Symbol']]
    ticker_list.append('SPY')
    data = yf.download(
            # tickers list or string as well
            tickers = ticker_list[0:],

            # use "period" instead of start/end
            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            # (optional, default is '1mo')
            period = "max",

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = "1d",

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            group_by = 'ticker',

            # adjust all OHLC automatically
            # (optional, default is False)
            auto_adjust = True,

            # download pre/post regular market hours data
            # (optional, default is False)
            prepost = True,

            # use threads for mass downloading? (True/False/Integer)
            # (optional, default is True)
            threads = True,

            # proxy URL scheme use use when downloading?
            # (optional, default is None)
            proxy = None
        )

    df = data.reindex(index=data.index[::-1])
    df = df.drop(['Open', 'High', 'Low', 'Volume'], axis=1, level=1)
    df.columns = df.columns.map(' '.join).str.strip()
    df.columns = [x.split()[0] for x in df.columns]
    df = df.sort_index(axis=0, ascending=True)

    def calc_corr(df=df,days=20,span=500):
        corr_dict = {}
        for stock in df.columns:
            for i in range(span):
                corr_dict[stock] = df.loc[:, stock].rolling(window=days).corr(df['SPY'])
        return corr_dict

    dic_20 = calc_corr(days=20, span=1)

    corr_df = pd.DataFrame(dic_20['ATVI'])
    for stock in dic_20:
        stock_df = pd.DataFrame(dic_20[stock])
        corr_df = pd.merge(corr_df, stock_df, left_index=True, right_index=True)

    corr_df.sort_index(ascending=False, inplace=True)    
        
    return corr_df.mean(axis=1)

def fred_pull():
    fred = Fred(api_key='')
    fred_df = pd.DataFrame()
    
    #Inflation breakevens
    fred_df['5yILBE'] = fred.get_series('T5YIE')
    fred_df['5y5yILBE'] = fred.get_series('T5YIFR')
    fred_df['10yILBE'] = fred.get_series('T10YIE')
    
    # Treasury rates
    fred_df['TedSpread'] = fred.get_series('TEDRATE')
    fred_df['FedFunds'] = fred.get_series('DFF')
    fred_df['2yTreas'] = fred.get_series('DGS2')
    fred_df['5yTreas'] = fred.get_series('DGS5')
    fred_df['10yTreas'] = fred.get_series('DGS10')
    fred_df['30yTreas'] = fred.get_series('DGS30')
    fred_df['5yrReal'] = fred_df['5yTreas'] - fred_df['5yILBE']
    fred_df['10yrReal'] = fred_df['10yTreas'] - fred_df['5yILBE']
    fred_df['Repo'] = fred.get_series('RRPONTSYD')
    
    #Other
    fred_df['WTI'] = fred.get_series('DCOILWTICO')
    fred_df['USDGBP'] = fred.get_series('DEXUSUK')
    fred_df['EURUSD'] = fred.get_series('DEXUSEU')
    fred_df['USDYUAN'] = fred.get_series('DEXCHUS')
    fred_df['USDYEN'] = fred.get_series('DEXJPUS')
    fred_df['NFCI'] = fred.get_series('NFCI')
    fred_df['NatGas'] = fred.get_series('DHHNGSP')
    fred_df['Mortgage'] = fred.get_series('MORTGAGE30US')
    fred_df['M1'] = fred.get_series('WM1NS')
    fred_df['M2'] = fred.get_series('WM2NS')
    fred_df['Desposits'] = fred.get_series('DPSACBW027SBOG')
    fred_df['Demand Deposits'] = fred.get_series('WDDNS')
    fred_df['C&I Loans'] = fred.get_series('TOTCI')
    fred_df['UMICH Sentiment'] = fred.get_series('UMCSENT')
    fred_df['UMICH Inflation'] = fred.get_series('MICH')
    fred_df['JOLTS'] = fred.get_series('JTSJOL')
    fred_df['Quits'] = fred.get_series('JTSQUR')
    
    # Curve feature engineering
    fred_df['2s10s'] = fred_df['10yTreas'] - fred_df['2yTreas']
    fred_df['2s30s'] = fred_df['30yTreas'] - fred_df['2yTreas']
    fred_df['5s10s'] = fred_df['10yTreas'] - fred_df['5yTreas']
    fred_df['10s30s'] = fred_df['30yTreas'] - fred_df['10yTreas']
    fred_df['5s30s'] = fred_df['30yTreas'] - fred_df['5yTreas']
    
    #Econ indicators
    fred_df['Labor_Force_Rate'] = fred.get_series('CIVPART')
    fred_df['Unemployment'] = fred.get_series('UNRATE')
    fred_df['Non-Farm Payrolls'] = fred.get_series('PAYEMS')
    
    fred_df = fred_df.fillna(method='ffill')
    
    return fred_df

def build_df():
    df = pull_tickers()
    vix_df = VVIX()
    df = pd.merge(df, vix_df, left_index=True, right_index=True, how='outer')
    df['SPX_RSI'] = RSI('^GSPC')
    df['SPX_20D_corr'] = spx_corr()
    df['SPX_20D_corr'] = df['SPX_20D_corr'].clip(0,1)
    df['SPX_20D_corr_delta'] = df['SPX_20D_corr'].diff(1)
    fred_df = fred_pull()
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_localize(None)
    fred_df.index = pd.to_datetime(fred_df.index)
    fred_df.index = fred_df.index.tz_localize(None)
    df = pd.merge(df, fred_df, left_index=True, right_index=True, how='outer')
    tech_df = tech_analysis_spx()
    tech_df.index = pd.to_datetime(tech_df.index)
    tech_df.index = tech_df.index.tz_localize(None)
    df = pd.merge(df, tech_df, left_index=True, right_index=True, how='outer')
    tenyr_tech_df = tech_analysis_10yr()
    tenyr_tech_df.index = pd.to_datetime(tenyr_tech_df.index)
    tenyr_tech_df.index = tenyr_tech_df.index.tz_localize(None)
    df = pd.merge(df, tenyr_tech_df, left_index=True, right_index=True, how='outer')
    df = df.sort_index(ascending=False)
    naaim = pd.read_excel('https://www.naaim.org/wp-content/uploads/2022/04/USE_Data-since-Inception_2022-4-06.xlsx').set_index('Date')
    df = pd.merge(df, naaim, left_index=True, right_index=True, how='outer')
    df = df.rename(columns={'Mean/Average': 'NAAIM'})
    df = df.fillna(method='ffill')
    df = df.sort_index(ascending=False)
    df['SPX_1D'] = df['SPX'].diff(1)/df['SPX']
    df['SPX_5D'] = df['SPX'].diff(5)/df['SPX']
    df['SPX_10D'] = df['SPX'].diff(10)/df['SPX']
    df['SPX_15D'] = df['SPX'].diff(15)/df['SPX']
    df['SPX_20D'] = df['SPX'].diff(20)/df['SPX']
    df['SPX_40D'] = df['SPX'].diff(40)/df['SPX']
    df['SPX_60D'] = df['SPX'].diff(60)/df['SPX']
    df['SPX_120D'] = df['SPX'].diff(120)/df['SPX']
    
    return df

pull_df = build_df()

windows = ['1d', '5d', 'SM', 'M']

for window in windows:

    df = pull_df.copy()
    df = df.resample(window).mean().sort_index(ascending = False)

    features = df.drop(['SPX_1D', 'SPX_5D', 'SPX_10D', 'SPX_15D', 'SPX_20D',
                        'SPX_40D', 'SPX_60D', 'SPX_120D'], axis=1).columns

    for feature in features:
        df[feature+'_chg1'] = df[feature][::-1].diff(1)[::1]
        df[feature+'_chg2'] = df[feature][::-1].diff(2)[::1]
        df[feature+'_chg3'] = df[feature][::-1].diff(3)[::1]

    df.to_csv(f'model_data_{window}.csv')

[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  504 of 504 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
import torch